# Urban Heat Islands and Street Trees

In this notebook we'll be exploring the urban heat island effect by looking at the impact on surface temperature of roof color and street trees. We'll be replicating the process described here: http://urbanspatialanalysis.com/urban-heat-islands-street-trees-in-philadelphia/ but using Python tools rather than ESRI.


**Extra packages:** To run this notebook, you'll need the PyViz tools and a library of *top of atmosphere* calculations from [`rio-toa`](https://github.com/mapbox/rio-toa): `pip install rio-toa`

**Data sources:** This notebook uses Landsat data from Google Cloud Storage as well as some geographic data from OpenDataPhilly.

In [ ]:
import intake
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gpd

import cartopy.crs as ccrs

import holoviews as hv
import hvplot.xarray
import hvplot.pandas

from geoviews.tile_sources import EsriImagery

hv.extension('bokeh')

In [ ]:
print('Minimum Versions:')
print('xarray:       ', '0.10.9')
print('geopandas:    ', '0.4.0')
print('intake:       ', '0.2.6')
print('intake-xarray:', '0.2.3')

Just some extra info about Landsat data:

In [ ]:
band_info = pd.DataFrame([
        (1,  "Aerosol", " 0.43 - 0.45",    0.440,  "30",         "Coastal aerosol"),
        (2,  "Blue",    " 0.45 - 0.51",    0.480,  "30",         "Blue"),
        (3,  "Green",   " 0.53 - 0.59",    0.560,  "30",         "Green"),
        (4,  "Red",     " 0.64 - 0.67",    0.655,  "30",         "Red"),
        (5,  "NIR",     " 0.85 - 0.88",    0.865,  "30",         "Near Infrared (NIR)"),
        (6,  "SWIR1",   " 1.57 - 1.65",    1.610,  "30",         "Shortwave Infrared (SWIR) 1"),
        (7,  "SWIR2",   " 2.11 - 2.29",    2.200,  "30",         "Shortwave Infrared (SWIR) 2"),
        (8,  "Panc",    " 0.50 - 0.68",    0.590,  "15",         "Panchromatic"),
        (9,  "Cirrus",  " 1.36 - 1.38",    1.370,  "30",         "Cirrus"),
        (10, "TIRS1",   "10.60 - 11.19",   10.895, "100 * (30)", "Thermal Infrared (TIRS) 1"),
        (11, "TIRS2",   "11.50 - 12.51",   12.005, "100 * (30)", "Thermal Infrared (TIRS) 2")],
    columns=['Band', 'Name', 'Wavelength Range (µm)', 'Nominal Wavelength (µm)', 'Resolution (m)', 'Description']).set_index(["Band"])
band_info

## Loading data

For this example, we will be using landsat data stored on Google Cloud Storage. Since these data are accessed via https, there is no guaranteed directory structure, so we will need to specify the url pointing to each file and then iterate over the files to create a concatenated dataset. We use jinja template notation in intake to pass parameters to the `urlpath`.

In [ ]:
cat = intake.open_catalog('../catalog.yml')
list(cat)

Let's take a look at what the `google_landsat_band` looks like:

In [ ]:
with open('../catalog.yml') as f:
    for line in f.readlines()[71:92]:
        print(line.rstrip())

The following might feel a bit arbitrary, but we have chosen the path and row corresponding to the area over Philadelphia using the [earth explorer](https://earthexplorer.usgs.gov/). We have also found the id of the particular date of interest using the same tool. With these values in hand, we can access parts of each file on Google Cloud Storage. 

In [ ]:
path = 14
row = 32
product_id = 'LC08_L1TP_014032_20160727_20170222_01_T1'

Loading in metadata from bands - note that bands themselves are not loaded in this cell (that's why it doesn't take very long). Only some file metadata and the coordinates are loaded. For this workflow we don't ever need to load all the data into memory or onto disk.

In [ ]:
# Load all the bands
# bands = range(1, 12)

# OR skip band 8 for now since it is at a different resolution and transform
bands = [1, 2, 3, 4, 5, 6, 7, 9, 10, 11]

datasets = []
for band in bands:
    da = cat.google_landsat_band(path=path, row=row, product_id=product_id, band=band).to_dask()
    da = da.assign_coords(band=[band])
    datasets.append(da)
    
ds = xr.concat(datasets, 'band', compat='identical')
print(ds)

Loading in metadata regarding these particular Landsat images from the associated matlab.txt file.

In [ ]:
def load_google_landsat_metadata(path, row, product_id):
    """Load landsat metdata for path, row, product_id from google cloud storage
    """
    def parse_type(x):
        try: 
            return eval(x)
        except:
            return x
    
    baseurl = 'https://storage.googleapis.com/gcp-public-data-landsat/LC08/01'
    suffix = f'{path:03d}/{row:03d}/{product_id}/{product_id}_MTL.txt'
    
    df = intake.open_csv(
        urlpath=f'{baseurl}/{suffix}',
        csv_kwargs={'sep': '=',
                    'header': None,
                    'names': ['index', 'value'],
                    'skiprows': 2,
                    'converters': {'index': (lambda x: x.strip()),
                                   'value': parse_type}}).read()
    metadata = df.set_index('index')['value']
    return metadata

In [ ]:
metadata = load_google_landsat_metadata(path, row, product_id)
metadata.head()

## Sub-setting to area of interest

So far we haven't downloaded any band data. Since we know that we are interested in Philadelphia, we can just take a smaller square of data that covers the extents of the city. First we need to know the projection of the dataset:

In [ ]:
ds.crs

We'll convert that into something more meaningful for later:

In [ ]:
crs = ccrs.epsg(32618)

Now if we were just looking for one particular point we could use that point, converted to the coordinate system of the data, and then select the data nearest to it:

In [ ]:
x_center, y_center = crs.transform_point(-75.1652, 39.9526, ccrs.PlateCarree())
nearest_to_center = ds.sel(x=x_center, y=y_center, method='nearest')
print(nearest_to_center.compute())

nearest_to_center.hvplot()

In this case, though, we are interested in a subset of data that covers that city of Philadelphia. So we need some geometry to specify the bounds of the city. We can get a GeoJSON of neighborhood data from [OpenDataPhilly](https://www.opendataphilly.org/dataset/philadelphia-neighborhoods/resource/6c61f240-aafe-478e-b993-b75fd09a93d6).

In [ ]:
url = 'https://github.com/azavea/geo-data/raw/master/Neighborhoods_Philadelphia/Neighborhoods_Philadelphia.geojson'
geoms = gpd.read_file(url)

The approach shown below is somewhat simplistic in that we will use the map to iteratively select the area of interest by tweaking the bounds out from the central lat, lon of interest. 

In [ ]:
subset = ds.sel(x=slice(x_center-1.2e4, x_center+1.9e4), y=slice(y_center+2.1e4, y_center-1.2e4))

In [ ]:
plot = subset.sel(band=3).hvplot(rasterize=True, crs=crs, height=500, width=600).redim(x="Longitude", y="Latitude")
plot.options(tools=[]) * geoms.hvplot(geo=True, alpha=.5, c='mapname')

Now that we have looked at our map and ensured that the area covers the city, we will just take this chunk of the bands to use for further analyses:

In [ ]:
ds = subset
ds

## Calculate NDVI

We'll calculate NDVI but we won't yet do any computations -- our bands are actually dask arrays, which allow for lazy computation.

In [ ]:
NDVI = (ds.sel(band=5) - ds.sel(band=4)) / (ds.sel(band=5) + ds.sel(band=4))
NDVI = NDVI.where(NDVI < np.inf)
NDVI

In order to visualize NDVI, the data will need to be loaded and the NDVI computed. We can expect this to take some non-trivial amount of time (on the order of 20 sec on a Macbook Pro laptop).

In [ ]:
p = NDVI.hvplot(datashade=True, x='x', y='y', crs=crs, height=500, width=600, cmap='viridis')

# We'll use a transparent rasterized version of the plot for hover text
p_hover = NDVI.hvplot(rasterize=True, x='x', y='y', crs=crs, height=500, width=600, alpha=0, colorbar=False)

p * p_hover

## Calculate land surface temperature

Given the NDVI calculated above, we can determine land surface temperature. For ease, we'll use some *top of atmosphere* calculations that have already been written up as Python functions as part of rasterio work in the `rio_toa` module. We'll also need to specify one more for transforming at satellite temperature (brightness temp) to land surface temperature. For these calculations we'll use both Thermal Infrared bands - 10 and 11.

In [ ]:
from rio_toa import brightness_temp, toa_utils

In [ ]:
def land_surface_temp(BT, w, NDVI):
    """Calculate land surface temperature of Landsat 8
    
    temp = BT/1 + w * (BT /p) * ln(e)
    
    BT = At Satellite temperature (brightness)
    w = wavelength of emitted radiance (μm)
    
    where p = h * c / s (1.439e-2 mK)
    
    h = Planck's constant (6.626e-34 Js)
    s = Boltzmann constant (1.38e-23 J/K)
    c = velocity of light (2.998e8 m/s)
    """
    h = 6.626e-34
    c = 2.998e8
    s = 1.38e-23
    
    p = (h * c / s) * 1e6
    
    Pv = (NDVI - NDVI.min() / NDVI.max() - NDVI.min())**2
    e = 0.004 * Pv + 0.986
    
    return BT / 1 + w * (BT / p) * np.log(e)

Now we'll set up a helper function to retrieve all the parameters from the metadata and general Landsat info table, and calculated the land surface temperature for band 10 and 11.

In [ ]:
def land_surface_temp_for_band(band):
    # params from general Landsat info
    w = band_info.loc[band]['Nominal Wavelength (µm)']
    
    # params from specific Landsat data text file for these images
    ML = metadata[f'RADIANCE_MULT_BAND_{band}']
    AL = metadata[f'RADIANCE_ADD_BAND_{band}']
    K1 = metadata[f'K1_CONSTANT_BAND_{band}']
    K2 = metadata[f'K2_CONSTANT_BAND_{band}']
    
    at_satellite_temp = brightness_temp.brightness_temp(ds.sel(band=band).values, ML, AL, K1, K2)
    
    return land_surface_temp(at_satellite_temp, w, NDVI)

In [ ]:
band = 10
temp_10 = land_surface_temp_for_band(band).compute()
temp_10_f = toa_utils.temp_rescale(temp_10, 'F')
band_10=ds.sel(band=band).copy(data=temp_10_f).hvplot(rasterize=True, x='x', y='y', cmap='fire_r', crs=crs, height=500, width=600)

band = 11
temp_11 = land_surface_temp_for_band(band).compute()
temp_11_f = toa_utils.temp_rescale(temp_11, 'F')
band_11=ds.sel(band=band).copy(data=temp_11_f).hvplot(rasterize=True, x='x', y='y', cmap='fire_r', crs=crs, height=500, width=600)

Compare the results from the two different bands, noticing that the colorbars are different.

In [ ]:
band_10.relabel('Band 10') + band_11.relabel('Band 11')

We'll take the mean of the calculated land surface temperature for each of the bands and mimic the colormap used in the project that we are duplicating.

In [ ]:
mean_temp = (temp_10 + temp_11) / 2
mean_temp_f = toa_utils.temp_rescale(mean_temp, 'F')

data = ds[0].copy(data=mean_temp_f)
p = (data
    .hvplot(rasterize=True, x='x', y='y', crs=crs, height=500, width=600, cmap='rainbow', alpha=.5, legend=False)
    .relabel('Mean Surface Temp (F)'))

p * EsriImagery 

Notice how the hot spots are located over warehouse roofs and parking lots. This becomes even more visible when just the temperatures greater than 90F are displayed. To show this, we'll make a special colormap that just includes high intensity reds that are found at the top of the `fire_r` colormap.

In [ ]:
import colorcet as cc

special_cmap = cc.fire[::-1][90:]

In [ ]:
data = ds[0].copy(data=mean_temp_f.where(mean_temp_f > 90))
thresholded_temp_p = (data
    .hvplot(x='x', y='y', cmap=special_cmap, crs=crs, height=500, width=500, colorbar=False, legend=False)
    .relabel('Mean Temp (F) > 90')
    .redim(value='Temperature (F)'))

thresholded_temp_p + thresholded_temp_p.options(alpha=.3) * EsriImagery

## Adding in the Street Tree data

OpenDataPhilly released an inventory of all the street trees in the city. Street trees are trees that are planted along streets, not those in parks and private property. Using `datashader` we can easily plot these 100,000 points.


It is hypothesized that where there are more trees the land surface temperature will be less extreme. To explore this, we will overlay street trees with the thresholded land surface temperature:

In [ ]:
url = 'http://data.phl.opendata.arcgis.com/datasets/957f032f9c874327a1ad800abd887d17_0.geojson'
trees = gpd.read_file(url)
trees.tail()

In [ ]:
tree_p = trees.hvplot(geo=True, datashade=True, height=500, width=500, legend=False, dynspread=30).relabel('Street Tree Density')

thresholded_temp_p.options(alpha=.5) * tree_p.options(alpha=.5)